In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision
import joblib
from PIL import Image
def model_activate(img_path,model_path):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = models.resnet18(weights=None).to(device)
  model.fc = nn.Sequential(
                nn.Linear(512, 128),
                nn.ReLU(inplace=True),
                nn.Linear(128, 2)).to(device)


  model = joblib.load(model_path)
  classes = ['others', 'monkeypox']
  #이건 테스트할 테스트 데이터 셋인데 저장해둔 사진 파일 위치 올려놓기

  file_list = Image.open(img_path)
  #위에 문장에서 파일명을 받았으니 파일명과 나머지 주소를 합치는 과정
  print(file_list)
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

  data_transforms = {
      'train':
      transforms.Compose([
          transforms.Resize((224,224)),
          transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          normalize
      ]),
      'test':
      transforms.Compose([
          transforms.Resize((224,224)),
          transforms.ToTensor(),
          normalize
      ]),
  }
  #이미지 전처리 과정인데 여기서 'test' 부분으로 test데이터셋 resize해서 이미지 크기 평준화 시키는곳
  validation_batch1 = torch.stack([data_transforms['test'](file_list).to(device)])

  #이건 내가 위에꺼가 원숭이두창, 나머지로 원숭이두창, 나머지의 결과가 어떻게 나오나 보려고 나눠둔것 validation_batch한개만 있어도 상관 없음
  pred_logits_tensor = model(validation_batch1)
  pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()
  monkey = int(pred_probs[0][0]*100)
  return monkey
  

In [14]:
monkey = model_activate("/content/drive/MyDrive/machine learning projects/training set/monkey pox/test/monkeypox/monkeypox83.png","/content/drive/MyDrive/machine learning projects/monkey pox/weight/resnet(cpu)(18).pkl")
print(monkey)

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=224x224 at 0x7F425B205A50>
98
